# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** *K*

**Names:**

* *Mathieu Sauser*
* *Luca Mouchel*
* *Jérémy Chaverot*
* *Heikel Jebali*

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [17]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix, save_npz
from utils import load_json, load_pkl
import json

import re
import pickle
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

[nltk_data] Downloading package punkt to /home/jchavero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jchavero/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Exercise 4.1: Pre-processing

In [18]:
# Maps each term to the number of times it appears
freqs = {}
# Map each course (cours['courseId']) to its processed description, i.e a list of words or n-grams
processedCourses = {}

for i, course in enumerate(courses):
    description = course['description']
    description = [char.lower() for char in description]
    description = ''.join(description)

    # Remove punctuation marks
    description = re.sub(r'[^\w\s]', '', description)

    # Tokenize the text into words
    tokens = nltk.word_tokenize(description)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords]                        
        
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    lemmatizedTokens = [lemmatizer.lemmatize(token) for token in tokens]
        
    # Get the frequency of each token and update the freqs map
    freqDist = FreqDist(lemmatizedTokens)
    for token in lemmatizedTokens:
        if token not in freqs.keys():
            freqs[token] = freqDist[token]
        else:
            freqs[token] += freqDist[token]

    # Add n-grams to the vocabulary
    nGramRange = (2, 3)  # Specify the range of n-grams to consider
    
    ngrams = []
    for i in range(nGramRange[0], nGramRange[1] + 1):
        ngrams.extend(list(nltk.ngrams(lemmatizedTokens, i)))
        
    ngrams = [' '.join(ngram) for ngram in ngrams]
    for ngram in ngrams:
        if ngram not in freqs.keys():
            freqs[ngram] = 1
        else:
            freqs[ngram] += 1
            
    vocabulary = lemmatizedTokens + ngrams
    
    processedCourses[course['courseId']] = vocabulary

In [19]:
# Find most frequent token
mostFreq = float('-inf')
mostFreqToken = ''
for token, freq in freqs.items():
    if freq > mostFreq: 
        mostFreq = freq
        mostFreqToken = token

print(f'**{mostFreqToken}** is the most used (lemmatized) word, with {mostFreq} apparitions')

**student** is the most used (lemmatized) word, with 9887 apparitions


In [20]:
freqWords = set([word for word in freqs.keys() if freqs[word] > mostFreq * 0.6])
infreqWords = set([word for word in freqs.keys() if freqs[word] < 3])

print(f'Most frequent words: {freqWords}')

Most frequent words: {'student', 'method', 'system'}


In [21]:
# Remove frequent and infrequent words and modify the freqs and processedCourses map accordingly
temp = {}
for term, freq in freqs.items():
    if freq < mostFreq and freq > 3:
        temp[term] = freq
        
freqs = temp

newProcessedCourses = {}
for courseId, description in processedCourses.items():
    newProcessedCourses[courseId] = [word for word in description if word in freqs.keys()]
    
    if courseId == 'COM-308':
        print(sorted(newProcessedCourses['COM-308']))

['20', '20 midterm', '30', '30 final', '30 final exam', '50', 'acquired', 'activity', 'activity lecture', 'ad', 'ad', 'algebra', 'algebra', 'algorithm', 'algorithm', 'algorithm data', 'algorithm data structure', 'analysis', 'analytics', 'analytics', 'application', 'application', 'assessment', 'assessment method', 'assessment method project', 'auction', 'auction', 'balance', 'based', 'based', 'basic', 'basic', 'basic', 'basic linear', 'basic linear algebra', 'basic material', 'basic model', 'cathedra', 'chain', 'class', 'class', 'class', 'cloud', 'clustering', 'clustering', 'collection', 'com300', 'combination', 'communication', 'communication com300', 'community', 'community', 'computing', 'computing', 'concept', 'concept', 'concept start', 'concrete', 'content', 'content class', 'course', 'course', 'course basic', 'course stochastic', 'course stochastic model', 'coverage', 'current', 'data', 'data', 'data', 'data', 'data', 'data', 'data mining', 'data mining', 'data mining', 'data str

In [22]:
# Save the pre-processed courses.txt file to disk
def get_preprocessed_courses(data): 
    with open('data/preprocessed_courses.txt', 'w') as f:
        for i in range(len(courses)):
            c = data[i]
            entry = {'courseId': c['courseId'],
                     'name': c['name'],
                     'description': newProcessedCourses[c['courseId']]}
            json.dump(entry, f)
            if i < len(courses) - 1: f.write(',\n')
            
preprocessed_courses = get_preprocessed_courses(courses)

- We removed the stopwords because they don't carry a lot of information for the meaning of the whole text and it will reduce the size of the TF-IDF matrix.
- We removed the punctuation for similar reasons.
- We removed very frequent words because they don't help us differentiate between documents if they appear everywhere
- We removed infrequent words because they don't represent the meaning of the text if they are not used often
- We lemmatized the words to group words with very similar meaning (e.g. 'chain' and chains')
- We added 2-grams and 3-grams to have additional information on how the words group together

## Exercise 4.2: Term-document matrix

In [23]:
# Maps each term to an index
termToIdx = {}
for i, term in enumerate(freqs.keys()):
    termToIdx[term] = i

# Maps each document to an index
docToIdx = {}
j = 0
for courseId, description in newProcessedCourses.items():
    docToIdx[courseId] = j
    j += 1

In [24]:
M = len(termToIdx.keys())
N = len(docToIdx.keys())
TD = np.zeros((M, N))

# Create the term-document matrix
for courseId, description in newProcessedCourses.items():
    for term in description:
        TD[termToIdx[term]][docToIdx[courseId]] += 1

# Get the number of terms for each document
termsPerDoc = np.sum(TD, axis=0)
# Compute TF and IDF to get TFIDF
TF = TD / termsPerDoc
IDF = np.log2(N / np.count_nonzero(TD, axis=1))
TFIDF = np.transpose(np.transpose(TF) * IDF)

In [25]:
np.save('./TFIDF.npy', TFIDF)
np.save('./termToIdx.npy', termToIdx)
np.save('./docToIdx.npy', docToIdx)

In [26]:
ixScores = np.argsort(-TFIDF[:, docToIdx['COM-308']])
top15idx = ixScores[:15]

print('------ Top 15 terms with highest TF-IDF scores in IX course ------\n')
print('------------------')
print('Term: Score')
print('------------------')
for idx in top15idx:
    for term, idx2 in termToIdx.items():
        if idx == idx2:
            print(f'{term}: {TFIDF[idx, docToIdx["COM-308"]]:.4f}')

------ Top 15 terms with highest TF-IDF scores in IX course ------

------------------
Term: Score
------------------
online: 0.0880
realworld: 0.0879
social: 0.0823
data mining: 0.0795
explore: 0.0764
mining: 0.0722
networking: 0.0687
hadoop: 0.0624
largescale: 0.0615
recommender: 0.0582
ecommerce: 0.0582
recommender system: 0.0582
service: 0.0559
auction: 0.0553
datasets: 0.0530


A high score means that the term appears a lot in the document but not that much in the rest of the rest of the corpus, meaning that it is really relevant for this given document.

A low score means that either the term is infrequent in the document and frequent everywhere, or infrequent everywhere, or very frequent everywhere hence reducing its relevance.

## Exercise 4.3: Document similarity search

In [27]:
from itertools import combinations

# Reverse the map of documents to index
idxToDoc = {v: k for k, v in docToIdx.items()}

In [28]:
def sim(i, j):
    '''
    Computes the similarity between two vectors using cosine distance
    
    Parameters:
        i (int): The index of the i-th document
        j (int): The index of the j-th document
        
    Returns:
        The similarity between the two documents
    '''
    d_i = np.array(TFIDF[:, i])
    d_j = np.array(TFIDF[:, j])
    
    return (d_i @ d_j) / (np.linalg.norm(d_i) * np.linalg.norm(d_j))

In [29]:
def docSimSearch(name):
    '''
    Finds the top 5 courses with the highest score for the term given
    
    Parameter:
        name (string): the name of the term
    '''
    courseIdx = termToIdx[name]
    courseTop5 = np.argsort(-TFIDF[courseIdx])[:5]
    combs = combinations(courseTop5, 2)

    simScores = []
    for comb in combs:
        i = comb[0]
        j = comb[1]
        simScores.append((sim(i, j), idxToDoc[i], idxToDoc[j]))

    print(f'----- Top five courses for "{name}" (Course: Score) ----- \n')
    for i, idx in enumerate(courseTop5):
        print(f'{i+1}. {idxToDoc[idx]}: {TFIDF[courseIdx, idx]:.4f}')
        
    print('\n----- Similarities between courses (Course 1 and Course 2: Similarity score) -----\n')
    for simScore in simScores:
        print(f'{simScore[1]} and {simScore[2]}: {simScore[0]:.4f}')

In [30]:
docSimSearch("markov chain")

----- Top five courses for "markov chain" (Course: Score) ----- 

1. MATH-332: 0.1890
2. COM-516: 0.1046
3. MGT-484: 0.0984
4. MATH-600: 0.0541
5. COM-512: 0.0304

----- Similarities between courses (Course 1 and Course 2: Similarity score) -----

MATH-332 and COM-516: 0.3214
MATH-332 and MGT-484: 0.2780
MATH-332 and MATH-600: 0.1557
MATH-332 and COM-512: 0.1229
COM-516 and MGT-484: 0.2719
COM-516 and MATH-600: 0.2126
COM-516 and COM-512: 0.2028
MGT-484 and MATH-600: 0.0900
MGT-484 and COM-512: 0.1113
MATH-600 and COM-512: 0.0288


We see that the top courses are all math-related courses or courses in which we use graph algorithms which makes sens for markov chains. The similarity between them is good, which also makes sense since they share topics like stochastic processes or graph algorithms

In [31]:
docSimSearch("facebook")

----- Top five courses for "facebook" (Course: Score) ----- 

1. EE-727: 0.1173
2. MICRO-453: 0.0000
3. BIO-629: 0.0000
4. CS-596: 0.0000
5. AR-482: 0.0000

----- Similarities between courses (Course 1 and Course 2: Similarity score) -----

EE-727 and MICRO-453: 0.0056
EE-727 and BIO-629: 0.0174
EE-727 and CS-596: 0.0162
EE-727 and AR-482: 0.0196
MICRO-453 and BIO-629: 0.0292
MICRO-453 and CS-596: 0.0136
MICRO-453 and AR-482: 0.0377
BIO-629 and CS-596: 0.0113
BIO-629 and AR-482: 0.0091
CS-596 and AR-482: 0.0251


Here we observe that the only course mentioning 'facebook' is EE-727. Obviously then, the similarity score between courses is very low since we don't really know how the 4 remaining courses were chosen.